# Regression

In [1]:
import geopandas as gpd

blocks_gdf = gpd.read_parquet('./../data/blocks.parquet')

In [2]:
from blocksnet.relations import generate_adjacency_graph

adjacency_graph = generate_adjacency_graph(blocks_gdf, 10)

2025-04-02 15:58:34.864 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_nodes:10 - Generating nodes
2025-04-02 15:58:34.873 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_edges:15 - Generating edges
2025-04-02 15:58:36.612 | SUCCESS  | blocksnet.relations.adjacency.core:generate_adjacency_graph:34 - Adjacency graph successfully generated: 16320 nodes, 16036 edges


## Density

In [3]:
from blocksnet.machine_learning.regression import DensityRegressor

dr = DensityRegressor()

### Train

In [4]:
data = dr.get_train_data(blocks_gdf, adjacency_graph)

2025-04-02 15:58:39.946 | WARNING  | blocksnet.machine_learning.regression.density.schemas:_before_validate:37 - Not valid format. Trying to one hot from land_use column
2025-04-02 15:58:43.172 | INFO     | blocksnet.preprocessing.feature_engineering.core:_calculate_usual_features:34 - Calculating usual features
2025-04-02 15:58:43.457 | INFO     | blocksnet.machine_learning.regression.density.core:_features_from_geometries:32 - Fitting the scaler


In [6]:
losses = dr.train(data, epochs=1000)

Current loss : 0.069: 100%|██████████| 1000/1000 [00:11<00:00, 89.14it/s]


### Evaluate

In [8]:
result_df = dr.evaluate(blocks_gdf, adjacency_graph)

2025-04-02 16:15:54.960 | WARNING  | blocksnet.machine_learning.regression.density.schemas:_before_validate:37 - Not valid format. Trying to one hot from land_use column


2025-04-02 16:15:57.825 | INFO     | blocksnet.preprocessing.feature_engineering.core:_calculate_usual_features:34 - Calculating usual features


In [9]:
result_df

,fsi,gsi,mxi
id,,,
0,0.098759,0.098759,0.272484
1,0.064455,0.064455,0.057871
2,0.158729,0.158729,0.216693
3,0.702841,0.256270,0.445032
4,0.062017,0.062017,0.240412
...,...,...,...
16315,0.004619,0.004619,0.076425
16316,0.003388,0.003388,0.019757
16317,0.017398,0.017398,0.080334


## Save model and scaler

In [12]:
dr.save_model('model.dict')